In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
import config

os.chdir('/home/brian/Documents/CPS/data/clean/')

In [3]:
# Code to update CPI as needed
api_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# API key in config.py which contains: bls_key = 'key'
key = '?registrationkey={}'.format(config.bls_key)

# Series stored as a dictionary
series_dict = {'CUSR0000SA0': 'ALL'}

# Start year and end year
date_r = (1994, 2018)

# Handle dates
dates = [(str(date_r[0]), str(date_r[1]))]
while int(dates[-1][1]) - int(dates[-1][0]) > 10:
    dates = [(str(date_r[0]), str(date_r[0]+9))]
    d1 = int(dates[-1][0])
    while int(dates[-1][1]) < date_r[1]:
        d1 = d1 + 10
        d2 = min([date_r[1], d1+9])
        dates.append((str(d1),(d2)))
        
df = pd.DataFrame()

for start, end in dates:
    # Submit the list of series as data
    data = json.dumps({
        "seriesid": list(series_dict.keys()),
        "startyear": start, "endyear": end})

    # Post request for the data
    p = requests.post(
        '{}{}'.format(api_url, key), 
        headers={'Content-type': 'application/json'}, 
        data=data).json()
    for s in p['Results']['series']:
        col = series_dict[s['seriesID']]
        for r in s['data']:
            date = pd.to_datetime('{} {}'.format(
                r['periodName'], r['year']))
            df.set_value(date, col, float(r['value']))
df = df.sort_index()
# Output results
print('Post Request Status: {}'.format(p['status']))

cpi = (df.iloc[-1] / df)
cpi.to_csv('cpi.csv')

In [21]:
cpi = pd.read_csv('cpi.csv', index_col=[0], parse_dates=True)

In [26]:
# Federal minimum wage--update monthly
fmw = pd.read_csv('fmw.csv', index_col=[0], parse_dates=True)

In [94]:
fed_min_wage = 7.25

data = (pd.read_feather('cps.ft', nthreads=3)
          .filter(items=['HRWAGE', 'PWORWGT', 'HRMONTH'])
          .query(f'HRWAGE >= {fed_min_wage}')
          .sort_values('HRWAGE')
          .assign(WAGE_RANGE = lambda x: pd.cut(x['HRWAGE'], 
                                                list(np.arange(0.125,200,0.25)), 
                                                include_lowest=True),
                  CS = lambda x: x['PWORWGT'].cumsum()))

midpt = data['PWORWGT'].sum() * 0.5

mid_interval = data.iloc[(data['CS']-midpt).abs().argsort()[:1]].WAGE_RANGE.values[0]

wage_bins = list(data['WAGE_RANGE'].unique())

mi_loc = wage_bins.index(mid_interval)
lowval = data[data['WAGE_RANGE'] == wage_bins[mi_loc-1]].iloc[-1].CS
highval = data[data['WAGE_RANGE'] == wage_bins[mi_loc]].iloc[-1].CS

binned_med = ((((midpt - lowval) / (highval - lowval)) * 0.5) + mid_interval.left)

print(f'binned, weighted median: ${binned_med: .2f}')

binned, weighted median: $ 18.93


/home/brian/anaconda3/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [92]:
data.iloc[(data['CS']-midpt).abs().argsort()[:1]].WAGE_RANGE.values[0]

Interval(18.875, 19.125, closed='right')

In [84]:
n = wage_bins.index(data.iloc[(data['CS']-midpt).abs().argsort()[:1]].WAGE_RANGE.values[0])
lowval = data[data['WAGE_RANGE'] == wage_bins[n-1]].iloc[-1].CS
highval = data[data['WAGE_RANGE'] == wage_bins[n]].iloc[-1].CS

binned_med = ((((midpt - lowval) / (highval - lowval)) * 0.5) + 
                  data.iloc[(data['CS']-midpt).abs().argsort()[:1]].WAGE_RANGE.values[0].left)

In [85]:
wage_bins[n-1]

Interval(18.625, 18.875, closed='right')

In [86]:
data.head()

,HRWAGE,PWORWGT,HRMONTH,WAGE_RANGE,CS
60000,7.25,3926.360840,2,"(7.125, 7.375]",3926.360840
480369,7.25,11001.754883,1,"(7.125, 7.375]",14928.115234
480374,7.25,18334.843750,1,"(7.125, 7.375]",33262.960938
673843,7.25,13278.716797,5,"(7.125, 7.375]",46541.679688
96954,7.25,15470.012695,2,"(7.125, 7.375]",62011.691406


In [88]:
lowval = data[data['WAGE_RANGE'] == wage_bins[n-1]].iloc[-1].CS
highval = data[data['WAGE_RANGE'] == wage_bins[n]].iloc[-1].CS

binned_med = ((((midpt - lowval) / (highval - lowval)) * 0.5) + 
                  data.iloc[(data['CS']-midpt).abs().argsort()[:1]].WAGE_RANGE.values[0].left)

In [89]:
binned_med = ((((midpt - lowval) / (highval - lowval)) * 0.5) + 
                  data.iloc[(data['CS']-midpt).abs().argsort()[:1]].WAGE_RANGE.values[0].left)

In [90]:
binned_med

18.92999832943535

In [68]:
data.iloc[(data['CS']-midpt).abs().argsort()[:1]].WAGE_RANGE.values[0]

Interval(18.875, 19.125, closed='right')

In [63]:
for month in list(data['HRMONTH'].unique()):
    df = data[data['HRMONTH'] == month]
    df['wage_range'] = pd.cut(df['HRWAGE'], list(np.arange(0.25,1000,0.5)), include_lowest=True)
    df = df.sort_values('HRWAGE')#.dropna(subset=['wage_range'])
    midpt = df['PWORWGT'].sum() * 0.5
    df['cs'] = df['PWORWGT'].cumsum()
    n = list(df['wage_range'].unique()).index(df.iloc[(df['cs']-midpt).abs().argsort()[:1]].wage_range.values[0])
    lowval = df[df['wage_range'] == list(df['wage_range'].unique())[n-1]].iloc[-1].cs
    highval = df[df['wage_range'] == list(df['wage_range'].unique())[n]].iloc[-1].cs
    binned_med = ((((midpt - lowval) / (highval - lowval)) * 0.5) + 
                  df.iloc[(df['cs']-midpt).abs().argsort()[:1]].wage_range.values[0].left)
    print(month)
    print(f'binned, weighted median: ${binned_med: .2f}')

/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

2
binned, weighted median: $ 18.79
1
binned, weighted median: $ 18.66
5
binned, weighted median: $ 18.75


/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

6
binned, weighted median: $ 18.47
4
binned, weighted median: $ 18.89
3
binned, weighted median: $ 18.77
7
binned, weighted median: $ 19.11
8
binned, weighted median: $ 18.68


/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
